In [ ]:
!pip install transformers[torch] accelerate

In [ ]:
import os
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import requests

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.io import read_image
from transformers import TrainingArguments, Trainer, MobileNetV2Config

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class CustomImageDataset(Dataset):
   # Inicjalizacja zestawu danych z katalogiem obrazów, procesorem i etykietami
    def __init__(self, img_dir, processor, labels):
        self.img_dir = img_dir
        self.img_labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):

        # Pobranie ścieżki do obrazu dla danego indeksu
        img_path = os.path.join(self.img_dir, self.img_labels[idx][0])
        # Wczytanie obrazu za pomocą funkcji read_image
        image = read_image(img_path)
        # Przetworzenie obrazu za pomocą określonego procesora i zwrócenie go jako tensory PyTorch. Tensory to struktury danych do przechowywania i przetwarzania danych numerycznych.
        processed = self.processor(images=image, return_tensors='pt')
        # Wyodrębnienie etykiety z zestawu danych i przekształcenie jej w tensor PyTorch
        label = torch.tensor(self.img_labels[idx][1])
        # Zwrócenie słownika zawierającego przetworzony obraz jako 'pixel_values' i etykietę jako 'labels'
        return {'pixel_values': processed['pixel_values'].squeeze(), 'labels': label}

In [ ]:
# Lista klas reprezentujących figury szachowe
class_names =  ['Rook', 'Queen','Pawn','Knight','King','Bishop']

# Słownik mapujący nazwy klas na ich odpowiadające indeksy
label_map = {class_name: i for i, class_name in enumerate(class_names)}

In [ ]:
# Klasy oraz przypisane im indeksy
label_map

{'Rook': 0, 'Queen': 1, 'Pawn': 2, 'Knight': 3, 'King': 4, 'Bishop': 5}

In [ ]:
# Katalog zawierający dane treningowe
img_dir = '/content/drive/MyDrive/Chess/Train'
# Pusta lista, która będzie przechowywać etykiety obrazów
img_labels = []

# Iteracja po słowniku label_map (nazwa klasy, indeks klasy)
for class_name, class_idx in label_map.items():
  # Ścieżka do katalogu z obrazami danej klasy
  class_dir = os.path.join(img_dir, class_name)
  # Iteracja po plikach obrazów w katalogu danej klasy
  for img_name in os.listdir(class_dir):
    # Dodanie krotki do listy img_labels zawierającej ścieżkę do obrazu i indeks klasy
    img_labels.append((os.path.join(class_name, img_name), class_idx))

In [ ]:
# Katalogi klas z przypisanymi indeksami
img_labels

[('Rook/00000035.jpg', 0),
 ('Rook/00000102.jpg', 0),
 ('Rook/00000075.jpg', 0),
 ('Rook/00000066.png', 0),
 ('Rook/00000021.jpg', 0),
 ('Rook/00000058.jpg', 0),
 ('Rook/00000015.jpg', 0),
 ('Rook/00000098.png', 0),
 ('Rook/00000044.jpg', 0),
 ('Rook/00000083.jpg', 0),
 ('Rook/00000020.jpg', 0),
 ('Rook/00000112.jpg', 0),
 ('Rook/00000099.jpg', 0),
 ('Rook/00000171.jpg', 0),
 ('Rook/00000031.jpg', 0),
 ('Rook/00000064.png', 0),
 ('Rook/00000063.jpg', 0),
 ('Rook/00000121.jpg', 0),
 ('Rook/00000117.jpg', 0),
 ('Rook/00000059.jpg', 0),
 ('Rook/00000056.png', 0),
 ('Rook/00000168.jpg', 0),
 ('Rook/00000016.jpg', 0),
 ('Rook/00000046.jpg', 0),
 ('Rook/00000026.png', 0),
 ('Rook/00000120.jpg', 0),
 ('Rook/00000142.jpg', 0),
 ('Rook/00000086.jpg', 0),
 ('Rook/00000107.jpg', 0),
 ('Rook/00000017.jpg', 0),
 ('Rook/00000022.jpg', 0),
 ('Rook/00000118.jpg', 0),
 ('Rook/00000034.jpg', 0),
 ('Rook/00000004.jpg', 0),
 ('Rook/00000054.jpg', 0),
 ('Rook/00000084.png', 0),
 ('Rook/00000180.jpg', 0),
 

In [ ]:
# Konfiguracja modelu MobileNetV2
config = MobileNetV2Config.from_pretrained('google/mobilenet_v2_1.0_224', num_labels=len(class_names))

# Mapowanie identyfikatorów klas na ich nazwy
config.id2label = {i: label for i, label in enumerate(class_names)}
# Mapowanie nazw klas na ich identyfikatory
config.label2id = {label: i for i, label in enumerate(class_names)}

In [ ]:
# Tworzenie procesora obrazów
processor = AutoImageProcessor.from_pretrained("google/mobilenet_v2_1.0_224")
# Tworzenie modelu MobileNetV2 dla klasyfikacji obrazów
model = AutoModelForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224", config=config, ignore_mismatched_sizes=True)

Some weights of MobileNetV2ForImageClassification were not initialized from the model checkpoint at google/mobilenet_v2_1.0_224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1001]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1001, 1280]) in the checkpoint and torch.Size([6, 1280]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tworzenie obiektu CustomImageDataset na podstawie zdefiniowanego img_dir, processor i img_labels
dataset = CustomImageDataset(img_dir, processor, img_labels)
# Tworzenie DataLoader do efektywnego przetwarzania danych treningowych w mini-batchach czyli podzbiorach danych treningowych, które są używane do aktualizacji wag modelu w procesie treningu.
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)
# Konfiguracja argumentów treningowych
training_arg = TrainingArguments(output_dir='results', num_train_epochs = 7)
# Tworzenie obiektu Trainer do przeprowadzenia procesu treningu
trainer = Trainer(model=model, args=training_arg, train_dataset=dataset)

In [ ]:
# Uruchamia proces treningu modelu
trainer.train()

Step,Training Loss
500,0.823000
1000,0.274300


TrainOutput(global_step=1001, training_loss=0.5482456025692609, metrics={'train_runtime': 2240.5914, 'train_samples_per_second': 3.568, 'train_steps_per_second': 0.447, 'total_flos': 1.6111681375297536e+16, 'train_loss': 0.5482456025692609, 'epoch': 7.0})

In [ ]:
model.save_pretrained('model_finetuned')
config.save_pretrained('model_finetuned')

In [ ]:
from sklearn.metrics import accuracy_score

# Ścieżka do katalogu z danymi testowymi
folder_path = '/content/drive/MyDrive/Chess/Test'

processor = AutoImageProcessor.from_pretrained('google/mobilenet_v2_1.0_224')
model = AutoModelForImageClassification.from_pretrained('model_finetuned')

# Listy przechowujące prawdziwe etykiety i przewidywane etykiety
true_labels = []
predicted_labels = []

# Lista przechowująca testowe straty
test_losses = []

# Iteracja po katalogach reprezentujących klasy w danych testowych
for class_dir in os.listdir(folder_path):
    class_path = os.path.join(folder_path, class_dir)
    if os.path.isdir(class_path):
        print(f"Przetwarzanie katalogu: {class_dir}")

        # Iteracja po plikach obrazów w katalogu danej klasy
        for image_file in os.listdir(class_path):
            if image_file.endswith(('.jpg', '.jpeg')):
                image_path = os.path.join(class_path, image_file)

                # Wczytanie obrazu i przetworzenie za pomocą procesora i modelu
                image = Image.open(image_path)
                inputs = processor(images=image, return_tensors="pt")
                outputs = model(**inputs)
                logits = outputs.logits
                predicted_class_idx = logits.argmax(-1).item()
                predicted_class = model.config.id2label[predicted_class_idx]

                # Pobranie prawdziwej etykiety z wcześniej utworzonego label_map
                true_label = label_map[class_dir]

                # Dodanie straty do listy
                loss = torch.nn.functional.cross_entropy(logits, torch.tensor([true_label]))
                test_losses.append(loss.item())

                # Aktualizacja list prawdziwych i przewidywanych etykiet
                true_labels.append(true_label)
                predicted_labels.append(predicted_class_idx)

                # Wyświetlenie obrazu z przewidzianą klasą
                plt.imshow(image)
                plt.title(predicted_class)
                plt.show()
                print(f"  Przewidziana klasa: {predicted_class}")

# Obliczenie dokładności
accuracy = accuracy_score(true_labels, predicted_labels)

# Obliczenie średniej testowej straty
average_test_loss = sum(test_losses) / len(test_losses) if len(test_losses) > 0 else 0.0

print("Accuracy:", accuracy)
print("Average Test Loss:", average_test_loss)


Output hidden; open in https://colab.research.google.com to view.